## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [ ]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [ ]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        #两层网络
        self.W1=tf.Variable(tf.random.normal(shape=[28*28,100]))
        self.b1=tf.Variable(tf.zeros(shape=[100]))
        self.W2=tf.Variable(tf.random.normal(shape=[100,10]))
        self.b2=tf.Variable(tf.zeros(shape=[10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        #化为一维
        x=tf.reshape(x,[-1,28*28])
        z1=tf.matmul(x,self.W1)+self.b1
        a1=tf.nn.relu(z1)
        z2=tf.matmul(a1,self.W2)+self.b2
        logits=z2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [14]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [16]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

ValueError: in user code:

    File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_21560\2382397481.py", line 15, in train_one_step  *
        logits = model(x)
    File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_21560\4253928374.py", line 17, in __call__  *
        z1=tf.matmul(x*self.W1)+self.b1

    ValueError: Dimensions must be equal, but are 60000 and 784 for '{{node mul}} = Mul[T=DT_FLOAT](Reshape, mul/ReadVariableOp)' with input shapes: [60000,784], [784,100].
